In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2
service = QiskitRuntimeService()  # přihlášení přes IBM Quantum API token
backend = service.least_busy(operational=True, simulator=False)
estimator = EstimatorV2(mode=backend)

from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator

# H2 Hamiltonian 2-qubit STO-3G
h2_paulis = [
    ("II", -1.052373245772859),
    ("ZI",  0.3979374248431808),
    ("IZ", -0.3979374248431808),
    ("ZZ", -0.01128010425623538),
    ("XX",  0.18093119978423156),
    ("YY",  0.18093119978423156)
]
h2_hamiltonian = SparsePauliOp.from_list(h2_paulis)

# Jednoduchý ansatz
ansatz = TwoLocal(num_qubits=2, rotation_blocks='ry', entanglement_blocks='cz', reps=1)

# VQE
vqe = VQE(estimator=Estimator(), ansatz=ansatz, optimizer=SLSQP())
vqe.initial_point = [0.0] * ansatz.num_parameters

result = vqe.compute_minimum_eigenvalue(h2_hamiltonian)

print(f"Electronic energy: {result.eigenvalue.real:.6f} Ha")

# Vizualizace ansatzu
ansatz.draw(output='mpl')



NameError: name 'service' is not defined

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2, Session
from qiskit_algorithms.optimizers import SLSQP
import numpy as np

# Přihlášení do IBM Quantum (musíš mít uložený API token)
service = QiskitRuntimeService()

# Vyber backend – můžeš zkusit reálný nebo simulátor
backend = service.least_busy(operational=True, simulator=False)

# Definice Hamiltoniánu H2 (2-qubit STO-3G)
h2_paulis = [
    ("II", -1.052373245772859),
    ("ZI",  0.3979374248431808),
    ("IZ", -0.3979374248431808),
    ("ZZ", -0.01128010425623538),
    ("XX",  0.18093119978423156),
    ("YY",  0.18093119978423156)
]
h2_hamiltonian = SparsePauliOp.from_list(h2_paulis)

# Ansatz
ansatz = TwoLocal(num_qubits=2, rotation_blocks="ry", entanglement_blocks="cz", reps=1)
num_params = ansatz.num_parameters

# Optimizer
optimizer = SLSQP(maxiter=100)

# VQE s EstimatorV2 – musíme ručně
def energy_eval(params, estimator):
    """Spočítá energii pro dané parametry."""
    qc = ansatz.bind_parameters(params)
    result = estimator.run([(qc, h2_hamiltonian)]).result()
    return result[0].data.evs[0]  # průměrná hodnota energie

# Spustíme session s EstimatorV2
with Session(service=service, backend=backend) as session:
    estimator = EstimatorV2(session=session)

    # Inicializace parametrů
    x0 = np.zeros(num_params)

    # Minimalizace energie
    res = optimizer.minimize(lambda p: energy_eval(p, estimator), x0)

    print(f"Optimal parameters: {res.x}")
    print(f"Ground state energy: {res.fun:.6f} Ha")

# Vizualizace ansatzu
ansatz.draw(output="mpl")
